In [1]:
# %load './otto_xgb.py'
"""
XGBoost + Count feature, Time: 1:37:22
XGBoost + Random indexing, Time: 1:56:58
The running times on i7 4790k, 32G MEM, GTX660
"""

import numpy as np
import scipy as sp
import pandas as pd

from sklearn.metrics import log_loss
import xgboost as xgb
from datetime import datetime

from utility import *

path = './'
file_train = path + 'train.csv'
file_test = path + 'test.csv'

training = pd.read_csv(file_train, index_col = 0)
test = pd.read_csv(file_test, index_col = 0)
num_train = training.shape[0]

target = training['target']
training.drop('target', inplace = True, axis = 1)

df_trans = pd.Series(range(9), index = target.unique())
y = target.map(df_trans).values
yMat = pd.get_dummies(y).values

X = np.vstack((training.values, test.values))


# Common parameters
nIter = 50
tc = 15 # max_depth
sh = .1 # eta
bf = .8 # subsample

### XGB1: Count feature
nt = 260
mb = 5 # min_child_weight
cs = 45. / 93 # colsample_bytree

X2, ignore = count_feature(X)
dtrain , dtest = xgb.DMatrix(X2[:num_train], label = y), xgb.DMatrix(X2[num_train:])

predAll_train = np.zeros((num_train, 9))
predAll_test = np.zeros((test.shape[0], 9))
scores = []

t0 = datetime.now()
for i in range(nIter):
    seed = i + 123
    param = {'bst:max_depth':tc, 'bst:eta':sh, 'silent':1, 'objective':'multi:softprob','num_class':9,
             'min_child_weight':mb, 'subsample':bf, 'colsample_bytree':cs, 'nthread':4, 'seed':seed,'eval_metric': 'mlogloss'}
    watchlist  = [(dtrain,'train')]
    plst = param.items()
    bst = xgb.train(plst, dtrain, nt,watchlist)
    # bst.save_model(path + 'model/model_XGB_CF_' + str(seed) + '.model')
    pred_train = bst.predict(dtrain).reshape((num_train, 9))
    pred_test = bst.predict(dtest).reshape(predAll_test.shape)
    predAll_train += pred_train
    predAll_test += pred_test
    sc1 = log_loss(yMat, pred_train)
    sc2 = log_loss(yMat, predAll_train / (i + 1))
    print (i, "  each:%f, avg:%f, Time:%s" % (sc1, sc2, datetime.now() - t0))

# 49   each:0.131026, avg:0.130512, Time:1:37:21.187174
pred_XGB_CF = predAll_test / nIter
np.save(path + 'pred_XGB_CF.npy', pred_XGB_CF)

### XGB2: Random Indexing
X1 = X / X.mean(0)

m = 140
k = 2
nt = 220
mb = 10 # min_child_weight
cs = 50. / 93 # colsample_bytree

predAll_train = np.zeros((num_train, 9))
predAll_test = np.zeros((test.shape[0], 9))
t0 = datetime.now()
for i in range(nIter):
    seed = i + 123210
    X3 = RI(X1, m, k, normalize = False, seed = seed)
    dtrain = xgb.DMatrix(X3[:num_train], label = y)
    dtest = xgb.DMatrix(X3[num_train:])
    watchlist  = [(dtrain,'train')]
    param = {'bst:max_depth':tc, 'bst:eta':sh, 'silent':1, 'objective':'multi:softprob',
             'num_class':9, 'min_child_weight':mb, 'subsample':bf,
             'colsample_bytree':cs, 'nthread':4, 'seed':seed}
    plst = param.items()
    bst = xgb.train(plst, dtrain, nt,watchlist)
    # bst.save_model(path + 'model/model_XGB_RI_' + str(seed) + '.model')
    pred_train = bst.predict(dtrain).reshape((num_train, 9))
    pred_test = bst.predict(dtest).reshape(predAll_test.shape)
    predAll_train += pred_train
    predAll_test += pred_test
    sc1 = log_loss(yMat, pred_train)
    sc2 = log_loss(yMat, predAll_train / (i + 1))
    print (i, "  each:%f, avg:%f, Time:%s" % (sc1, sc2, datetime.now() - t0))

# 49   each:0.085345, avg:0.084942, Time:1:56:57.580157
pred_XGB_RI = predAll_test / nIter
np.save(path + 'pred_XGB_RI.npy', pred_XGB_RI)

[0]	train-mlogloss:1.980793
[1]	train-mlogloss:1.817169
[2]	train-mlogloss:1.669235
[3]	train-mlogloss:1.549015
[4]	train-mlogloss:1.445978
[5]	train-mlogloss:1.357949
[6]	train-mlogloss:1.284582
[7]	train-mlogloss:1.216188
[8]	train-mlogloss:1.151986
[9]	train-mlogloss:1.093660
[10]	train-mlogloss:1.040376
[11]	train-mlogloss:0.991099
[12]	train-mlogloss:0.947887
[13]	train-mlogloss:0.907467
[14]	train-mlogloss:0.869320
[15]	train-mlogloss:0.835869
[16]	train-mlogloss:0.804082
[17]	train-mlogloss:0.773739
[18]	train-mlogloss:0.748522
[19]	train-mlogloss:0.721629
[20]	train-mlogloss:0.697873
[21]	train-mlogloss:0.676393
[22]	train-mlogloss:0.655483
[23]	train-mlogloss:0.635832
[24]	train-mlogloss:0.616998
[25]	train-mlogloss:0.599815
[26]	train-mlogloss:0.582867
[27]	train-mlogloss:0.568260
[28]	train-mlogloss:0.553007
[29]	train-mlogloss:0.539559
[30]	train-mlogloss:0.526234
[31]	train-mlogloss:0.514497
[32]	train-mlogloss:0.502715
[33]	train-mlogloss:0.491708
[34]	train-mlogloss:0.48

0   each:0.130992, avg:0.130992, Time:0:05:03.244352


[8]	train-mlogloss:1.143354


KeyboardInterrupt: 

In [ ]:
# %load 'otto_xgb.py'
"""
XGBoost + Count feature, Time: 1:37:22
XGBoost + Random indexing, Time: 1:56:58
The running times on i7 4790k, 32G MEM, GTX660
"""

import numpy as np
import scipy as sp
import pandas as pd

from sklearn.metrics import log_loss
import xgboost as xgb
from datetime import datetime

from utility import *

path = './'
file_train = path + 'train.csv'
file_test = path + 'test.csv'

training = pd.read_csv(file_train, index_col = 0)
test = pd.read_csv(file_test, index_col = 0)
num_train = training.shape[0]

target = training['target']
training.drop('target', inplace = True, axis = 1)

df_trans = pd.Series(range(9), index = target.unique())
y = target.map(df_trans).values
yMat = pd.get_dummies(y).values

X = np.vstack((training.values, test.values))


# Common parameters
nIter = 50
tc = 15 # max_depth
sh = .1 # eta
bf = .8 # subsample

### XGB1: Count feature
nt = 260
mb = 5 # min_child_weight
cs = 45. / 93 # colsample_bytree

X2, ignore = count_feature(X)
dtrain , dtest = xgb.DMatrix(X2[:num_train], label = y), xgb.DMatrix(X2[num_train:])

predAll_train = np.zeros((num_train, 9))
predAll_test = np.zeros((test.shape[0], 9))
scores = []

t0 = datetime.now()
for i in range(nIter):
    seed = i + 123
    param = {'bst:max_depth':tc, 'bst:eta':sh, 'silent':1, 'objective':'multi:softprob','num_class':9,
             'min_child_weight':mb, 'subsample':bf, 'colsample_bytree':cs, 'nthread':8, 'seed':seed}
    plst = param.items()
    bst = xgb.train(plst, dtrain, nt)
    # bst.save_model(path + 'model/model_XGB_CF_' + str(seed) + '.model')
    pred_train = bst.predict(dtrain).reshape((num_train, 9))
    pred_test = bst.predict(dtest).reshape(predAll_test.shape)
    predAll_train += pred_train
    predAll_test += pred_test
    sc1 = log_loss(yMat, pred_train)
    sc2 = log_loss(yMat, predAll_train / (i + 1))
    print i, "  each:%f, avg:%f, Time:%s" % (sc1, sc2, datetime.now() - t0)

# 49   each:0.131026, avg:0.130512, Time:1:37:21.187174
pred_XGB_CF = predAll_test / nIter
np.save(path + 'pred_XGB_CF.npy', pred_XGB_CF)

### XGB2: Random Indexing
X1 = X / X.mean(0)

m = 140
k = 2
nt = 220
mb = 10 # min_child_weight
cs = 50. / 93 # colsample_bytree

predAll_train = np.zeros((num_train, 9))
predAll_test = np.zeros((test.shape[0], 9))
t0 = datetime.now()
for i in range(nIter):
    seed = i + 123210
    X3 = RI(X1, m, k, normalize = False, seed = seed)
    dtrain = xgb.DMatrix(X3[:num_train], label = y)
    dtest = xgb.DMatrix(X3[num_train:])
    param = {'bst:max_depth':tc, 'bst:eta':sh, 'silent':1, 'objective':'multi:softprob',
             'num_class':9, 'min_child_weight':mb, 'subsample':bf,
             'colsample_bytree':cs, 'nthread':8, 'seed':seed}
    plst = param.items()
    bst = xgb.train(plst, dtrain, nt)
    # bst.save_model(path + 'model/model_XGB_RI_' + str(seed) + '.model')
    pred_train = bst.predict(dtrain).reshape((num_train, 9))
    pred_test = bst.predict(dtest).reshape(predAll_test.shape)
    predAll_train += pred_train
    predAll_test += pred_test
    sc1 = log_loss(yMat, pred_train)
    sc2 = log_loss(yMat, predAll_train / (i + 1))
    print i, "  each:%f, avg:%f, Time:%s" % (sc1, sc2, datetime.now() - t0)

# 49   each:0.085345, avg:0.084942, Time:1:56:57.580157
pred_XGB_RI = predAll_test / nIter
np.save(path + 'pred_XGB_RI.npy', pred_XGB_RI)


In [2]:
import numpy as np
import scipy as sp
import pandas as pd

from sklearn.metrics import log_loss
import xgboost as xgb
from datetime import datetime

from utility import *

In [30]:
from __future__ import print_function
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss
from sklearn import preprocessing

import numpy as np
import pandas as pd

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import xgboost as xgb


def load_train():
    train = pd.read_csv('./train.csv')
    labels = train.target.values
    lbl_enc = preprocessing.LabelEncoder()
    labels = lbl_enc.fit_transform(labels)
    train = train.drop('id', axis=1)
    train = train.drop('target', axis=1)
    return train.values, labels.astype('int32')


def load_test():
    test = pd.read_csv('./test.csv')
    test = test.drop('id', axis=1)
    return test.values


def write_submission(preds, output):
    sample = pd.read_csv('./data/sampleSubmission.csv')
    preds = pd.DataFrame(
        preds, index=sample.id.values, columns=sample.columns[1:])
    preds.to_csv(output, index_label='id')


def score(params):
    print ("Training with params : ")
    print (params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain, 'train'),(dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round,watchlist,early_stopping_rounds=10)
    predictions = model.predict(dvalid).reshape((X_test.shape[0], 9))
    score = log_loss(y_test, predictions)
    print ("\tScore {0}\n\n".format(score))
    return {'loss': score, 'status': STATUS_OK}


def optimize(trials):
    space = {
             'n_estimators' : 500,
             'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
             'max_depth' : hp.quniform('max_depth', 1, 13, 1),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
             'num_class' : 9,
             'eval_metric': 'mlogloss',
             'objective': 'multi:softprob',
             'nthread' : 8,
             'silent' : 1
             }

    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=250)

    print (best)

In [6]:
X, y = load_train()
print ("Splitting data into train and valid ...\n\n")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234)

Splitting data into train and valid ...




In [11]:
tc = 15 # max_depth
sh = .1 # eta
bf = .8 # subsample

### XGB1: Count feature
nt = 260
mb = 5 # min_child_weight
cs = 45. / 93 # colsample_bytree

In [31]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)

In [18]:
param = {'bst:max_depth':tc, 'bst:eta':sh, 'silent':1, 'objective':'multi:softprob',
             'num_class':9, 'min_child_weight':mb, 'subsample':bf,
             'colsample_bytree':cs, 'nthread':8,'eval_metric': 'mlogloss', 'seed':1}

In [22]:
plst = param.items()
bst = xgb.train(plst, dtrain, nt,evals=[(dtrain, 'train'),(dvalid, 'eval')])

Will train until train error hasn't decreased in 3 rounds.
[0]	eval-mlogloss:1.986198	train-mlogloss:1.975497
[1]	eval-mlogloss:1.821582	train-mlogloss:1.801462
[2]	eval-mlogloss:1.697675	train-mlogloss:1.666939
[3]	eval-mlogloss:1.579083	train-mlogloss:1.540579
[4]	eval-mlogloss:1.486656	train-mlogloss:1.441033
[5]	eval-mlogloss:1.404819	train-mlogloss:1.352361
[6]	eval-mlogloss:1.333753	train-mlogloss:1.274717
[7]	eval-mlogloss:1.267014	train-mlogloss:1.201703
[8]	eval-mlogloss:1.210053	train-mlogloss:1.138829
[9]	eval-mlogloss:1.157060	train-mlogloss:1.080439
[10]	eval-mlogloss:1.110415	train-mlogloss:1.027890
[11]	eval-mlogloss:1.064708	train-mlogloss:0.976980
[12]	eval-mlogloss:1.027565	train-mlogloss:0.935157
[13]	eval-mlogloss:0.990397	train-mlogloss:0.892800
[14]	eval-mlogloss:0.959084	train-mlogloss:0.856402
[15]	eval-mlogloss:0.928589	train-mlogloss:0.821732
[16]	eval-mlogloss:0.901501	train-mlogloss:0.790498
[17]	eval-mlogloss:0.874847	train-mlogloss:0.759575
[18]	eval-mlogl

In [14]:
xgb.train?

In [29]:
X, y = load_train()
print ("Splitting data into train and valid ...\n\n")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234)

#Trials object where the history of search will be stored
trials = Trials()

optimize(trials)

Splitting data into train and valid ...


Training with params : 

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.491517	eval-mlogloss:1.492735
[1]	train-mlogloss:1.294912	eval-mlogloss:1.300167
[2]	train-mlogloss:1.178109	eval-mlogloss:1.183033
[3]	train-mlogloss:1.084187	eval-mlogloss:1.089740
[4]	train-mlogloss:1.016327	eval-mlogloss:1.022353
[5]	train-mlogloss:0.967663	eval-mlogloss:0.975172
[6]	train-mlogloss:0.923306	eval-mlogloss:0.931761
[7]	train-mlogloss:0.887959	eval-mlogloss:0.896196
[8]	train-mlogloss:0.860368	eval-mlogloss:0.868892
[9]	train-mlogloss:0.836863	eval-mlogloss:0.844343
[10]	train-mlogloss:0.816982	eval-mlogloss:0.824781
[11]	train-mlogloss:0.798980	eval-mlogloss:0.807633
[12]	train-mlogloss:0.784160	eval-mlogloss:0.792670
[13]	train-mlogloss:0.770862	eval-mlogloss:0.779892
[14]	train-mlogloss:0.758900	eval-mlogloss:0.767076
[15]	train-mlogloss:0.746889	eval-mlogloss:0.754389
[16]	train-mlogloss:0.737020	eval-mlogloss:0.744447
[17]	train-mlogloss:0.727481	eval-mlogloss:0.735359
[18]	train-mlog


{'eta': 0.5, 'eval_metric': 'mlogloss', 'colsample_bytree': 0.9, 'max_depth': 2.0, 'silent': 1, 'subsample': 0.65, 'num_class': 9, 'min_child_weight': 4.0, 'n_estimators': 765.0, 'objective': 'multi:softprob', 'nthread': 8, 'gamma': 0.8500000000000001}
	Score 0.517149685994843



Stopping. Best iteration:
[358]	train-mlogloss:0.407554	eval-mlogloss:0.516860

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.387802	eval-mlogloss:1.424292
[1]	train-mlogloss:1.099197	eval-mlogloss:1.162659
[2]	train-mlogloss:0.908458	eval-mlogloss:0.993347
[3]	train-mlogloss:0.768930	eval-mlogloss:0.875547
[4]	train-mlogloss:0.667061	eval-mlogloss:0.793097
[5]	train-mlogloss:0.589284	eval-mlogloss:0.733462
[6]	train-mlogloss:0.530940	eval-mlogloss:0.685944
[7]	train-mlogloss:0.487268	eval-mlogloss:0.652298
[8]	train-mlogloss:0.448896	eval-mlogloss:0.623722
[9]	train-mlogloss:0.416490	eval-mlogloss:0.601429
[10]	train-mlogloss:0.390155	eval-mlogloss:0.583237
[11]	train-mlogloss:0.368068	eval-mlogloss:0.568567
[12]	train-mlogloss:0.348025	eval-mlogloss:0.555780
[13]	train-mlogloss:0.334299	eval-mlogloss:0.546531
[14]	train-mlogloss:0.319369	eval-mlogloss:0.538504
[15]	train-mlogloss:0.307026	eval-mlogloss:0.530754
[16]	train-mlogloss:0.296503	eval-mlogl


Training with params : 
{'eta': 0.35000000000000003, 'eval_metric': 'mlogloss', 'colsample_bytree': 0.8500000000000001, 'max_depth': 13.0, 'silent': 1, 'subsample': 0.8500000000000001, 'num_class': 9, 'min_child_weight': 4.0, 'n_estimators': 424.0, 'objective': 'multi:softprob', 'nthread': 8, 'gamma': 0.6000000000000001}
	Score 0.47931245458101845



...

[640]	train-mlogloss:0.497829	eval-mlogloss:0.539407
Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.661219	eval-mlogloss:1.687792
[1]	train-mlogloss:1.385520	eval-mlogloss:1.428874
[2]	train-mlogloss:1.189140	eval-mlogloss:1.245925
[3]	train-mlogloss:1.044403	eval-mlogloss:1.111474
[4]	train-mlogloss:0.931804	eval-mlogloss:1.009096
[5]	train-mlogloss:0.840799	eval-mlogloss:0.928516
[6]	train-mlogloss:0.765589	eval-mlogloss:0.862888
[7]	train-mlogloss:0.703140	eval-mlogloss:0.808959
[8]	train-mlogloss:0.651571	eval-mlogloss:0.766086
[9]	train-mlogloss:0.607486	eval-mlogloss:0.729013
[10]	train-mlogloss:0.569969	eval-mlogloss:0.697869
[11]	train-mlogloss:0.539300	eval-mlogloss:0.672477
[12]	train-mlogloss:0.511660	eval-mlogloss:0.651001
[13]	train-mlogloss:0.487413	eval-mlogloss:0.632473
[14]	train-mlogloss:0.465773	eval-mlogloss:0.616627
[15]	train-mlogloss:0.445567	eval-mlogloss:0.602452
[16]	train-mlogloss:0.428869	eval-mlogloss:0.590352
[17]	train-mlo


Training with params : 
{'eta': 0.225, 'eval_metric': 'mlogloss', 'colsample_bytree': 0.9, 'max_depth': 10.0, 'silent': 1, 'subsample': 0.9500000000000001, 'num_class': 9, 'min_child_weight': 2.0, 'n_estimators': 819.0, 'objective': 'multi:softprob', 'nthread': 8, 'gamma': 0.8500000000000001}
	Score 0.46714755656078166



Stopping. Best iteration:
[149]	train-mlogloss:0.146386	eval-mlogloss:0.466887

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:2.095997	eval-mlogloss:2.096916
[1]	train-mlogloss:2.016969	eval-mlogloss:2.018269
[2]	train-mlogloss:1.939236	eval-mlogloss:1.940912
[3]	train-mlogloss:1.875059	eval-mlogloss:1.877060
[4]	train-mlogloss:1.814607	eval-mlogloss:1.816700
[5]	train-mlogloss:1.762323	eval-mlogloss:1.764922
[6]	train-mlogloss:1.713485	eval-mlogloss:1.716723
[7]	train-mlogloss:1.661674	eval-mlogloss:1.665639
[8]	train-mlogloss:1.614648	eval-mlogloss:1.619224
[9]	train-mlogloss:1.570102	eval-mlogloss:1.575079
[10]	train-mlogloss:1.529675	eval-mlogloss:1.534919
[11]	train-mlogloss:1.493852	eval-mlogloss:1.499604
[12]	train-mlogloss:1.460596	eval-mlogloss:1.466644
[13]	train-mlogloss:1.425658	eval-mlogloss:1.432446
[14]	train-mlogloss:1.392177	eval-mlogloss:1.399280
[15]	train-mlogloss:1.361519	eval-mlogloss:1.368998
[16]	train-mlogloss:1.332784	eval-mlogl


Training with params : 
{'eta': 0.05, 'eval_metric': 'mlogloss', 'colsample_bytree': 0.55, 'max_depth': 5.0, 'silent': 1, 'subsample': 0.75, 'num_class': 9, 'min_child_weight': 3.0, 'n_estimators': 665.0, 'objective': 'multi:softprob', 'nthread': 8, 'gamma': 0.8500000000000001}


KeyboardInterrupt: 

In [32]:
X, y = load_train()
print ("Splitting data into train and valid ...\n\n")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234)

#Trials object where the history of search will be stored
trials = Trials()

optimize(trials)

Splitting data into train and valid ...


Training with params : 

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.613063	eval-mlogloss:1.635780
[1]	train-mlogloss:1.340728	eval-mlogloss:1.385864
[2]	train-mlogloss:1.150567	eval-mlogloss:1.210334
[3]	train-mlogloss:1.007936	eval-mlogloss:1.080255
[4]	train-mlogloss:0.899594	eval-mlogloss:0.982222
[5]	train-mlogloss:0.811950	eval-mlogloss:0.904010
[6]	train-mlogloss:0.740421	eval-mlogloss:0.841204
[7]	train-mlogloss:0.676811	eval-mlogloss:0.787053
[8]	train-mlogloss:0.629190	eval-mlogloss:0.746192
[9]	train-mlogloss:0.588885	eval-mlogloss:0.713217
[10]	train-mlogloss:0.550324	eval-mlogloss:0.681779
[11]	train-mlogloss:0.519391	eval-mlogloss:0.657543
[12]	train-mlogloss:0.492662	eval-mlogloss:0.636952
[13]	train-mlogloss:0.471745	eval-mlogloss:0.620226
[14]	train-mlogloss:0.449592	eval-mlogloss:0.604870
[15]	train-mlogloss:0.428937	eval-mlogloss:0.590580
[16]	train-mlogloss:0.412923	eval-mlogloss:0.578653
[17]	train-mlogloss:0.399348	eval-mlogloss:0.569375
[18]	train-mlog


{'eta': 0.25, 'eval_metric': 'mlogloss', 'colsample_bytree': 0.65, 'max_depth': 12.0, 'silent': 1, 'subsample': 0.9500000000000001, 'num_class': 9, 'min_child_weight': 5.0, 'n_estimators': 500, 'objective': 'multi:softprob', 'nthread': 8, 'gamma': 0.8}
	Score 0.4658277153093069



Stopping. Best iteration:
[130]	train-mlogloss:0.147622	eval-mlogloss:0.465313

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.873075	eval-mlogloss:1.876454
[1]	train-mlogloss:1.690907	eval-mlogloss:1.690414
[2]	train-mlogloss:1.572723	eval-mlogloss:1.572101
[3]	train-mlogloss:1.486662	eval-mlogloss:1.486391
[4]	train-mlogloss:1.419091	eval-mlogloss:1.418536
[5]	train-mlogloss:1.361313	eval-mlogloss:1.360339
[6]	train-mlogloss:1.313528	eval-mlogloss:1.312695
[7]	train-mlogloss:1.273811	eval-mlogloss:1.274210
[8]	train-mlogloss:1.238327	eval-mlogloss:1.239969
[9]	train-mlogloss:1.208471	eval-mlogloss:1.209948
[10]	train-mlogloss:1.180933	eval-mlogloss:1.182281
[11]	train-mlogloss:1.157274	eval-mlogloss:1.159661
[12]	train-mlogloss:1.135773	eval-mlogloss:1.138808
[13]	train-mlogloss:1.115140	eval-mlogloss:1.119230
[14]	train-mlogloss:1.097535	eval-mlogloss:1.101810
[15]	train-mlogloss:1.080963	eval-mlogloss:1.085578
[16]	train-mlogloss:1.065539	eval-mlogl


Training with params : 
{'eta': 0.30000000000000004, 'eval_metric': 'mlogloss', 'colsample_bytree': 0.9500000000000001, 'max_depth': 1.0, 'silent': 1, 'subsample': 0.5, 'num_class': 9, 'min_child_weight': 4.0, 'n_estimators': 500, 'objective': 'multi:softprob', 'nthread': 8, 'gamma': 0.6000000000000001}
	Score 0.5789096562381619



...


Training with params : 
{'eta': 0.05, 'eval_metric': 'mlogloss', 'colsample_bytree': 0.7000000000000001, 'max_depth': 10.0, 'silent': 1, 'subsample': 0.9500000000000001, 'num_class': 9, 'min_child_weight': 4.0, 'n_estimators': 500, 'objective': 'multi:softprob', 'nthread': 8, 'gamma': 0.7000000000000001}
	Score 0.45482581334986477



Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.815709	eval-mlogloss:1.830334
[1]	train-mlogloss:1.583796	eval-mlogloss:1.608976
[2]	train-mlogloss:1.411073	eval-mlogloss:1.445285
[3]	train-mlogloss:1.273646	eval-mlogloss:1.315489
[4]	train-mlogloss:1.162207	eval-mlogloss:1.212114
[5]	train-mlogloss:1.068211	eval-mlogloss:1.125213
[6]	train-mlogloss:0.989877	eval-mlogloss:1.053204
[7]	train-mlogloss:0.921431	eval-mlogloss:0.991018
[8]	train-mlogloss:0.861357	eval-mlogloss:0.937741
[9]	train-mlogloss:0.809256	eval-mlogloss:0.891802
[10]	train-mlogloss:0.763579	eval-mlogloss:0.851779
[11]	train-mlogloss:0.723253	eval-mlogloss:0.816997
[12]	train-mlogloss:0.686467	eval-mlogloss:0.785535
[13]	train-mlogloss:0.654103	eval-mlogloss:0.758343
[14]	train-mlogloss:0.625567	eval-mlogloss:0.734698



Training with params : 
{'eta': 0.15000000000000002, 'eval_metric': 'mlogloss', 'colsample_bytree': 0.9, 'max_depth': 11.0, 'silent': 1, 'subsample': 0.65, 'num_class': 9, 'min_child_weight': 3.0, 'n_estimators': 500, 'objective': 'multi:softprob', 'nthread': 8, 'gamma': 0.9}


[15]	train-mlogloss:0.599389	eval-mlogloss:0.713576


KeyboardInterrupt: 

In [ ]:
{'eta': 0.05, 'eval_metric': 'mlogloss', 'colsample_bytree': 0.5,
 'max_depth': 12.0, 'silent': 1, 'subsample': 0.8500000000000001,
 'num_class': 9, 'min_child_weight': 2.0, 'n_estimators': 500, 
 'objective': 'multi:softprob', 'nthread': 8, 'gamma': 0.7000000000000001}

In [37]:
def score(params):
    print ("Training with params : ")
    print (params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain, 'train'),(dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round,watchlist,early_stopping_rounds=10)
    predictions = model.predict(dvalid).reshape((X_test.shape[0], 9))
    score = log_loss(y_test, predictions)
    print ("\tScore {0}\n\n".format(score))
    return {'loss': score, 'status': STATUS_OK}


def optimize(trials):
    space = {
             'n_estimators' : 1000,
             'eta' : 0.05,
             'max_depth' : hp.quniform('max_depth', 9, 13, 1),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : 0.85,
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : 0.5,
             'num_class' : 9,
             'eval_metric': 'mlogloss',
             'objective': 'multi:softprob',
             'nthread' : 8,
             'silent' : 1
             }

    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=250)

    print (best)

In [35]:
trials.best_trial

{'book_time': datetime.datetime(2015, 9, 11, 13, 41, 59, 903000),
 'exp_key': None,
 'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'idxs': {'colsample_bytree': [24],
   'eta': [24],
   'gamma': [24],
   'max_depth': [24],
   'min_child_weight': [24],
   'subsample': [24]},
  'tid': 24,
  'vals': {'colsample_bytree': [0.5],
   'eta': [0.050000000000000003],
   'gamma': [0.70000000000000007],
   'max_depth': [12.0],
   'min_child_weight': [2.0],
   'subsample': [0.85000000000000009]},
  'workdir': None},
 'owner': None,
 'refresh_time': datetime.datetime(2015, 9, 11, 13, 44, 6, 917000),
 'result': {'loss': 0.4489965501643818, 'status': 'ok'},
 'spec': None,
 'state': 2,
 'tid': 24,
 'version': 0}

In [ ]:
trials = Trials()

optimize(trials)

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:2.085175	eval-mlogloss:2.090216
[1]	train-mlogloss:1.981482	eval-mlogloss:1.991282
[2]	train-mlogloss:1.891570	eval-mlogloss:1.905004
[3]	train-mlogloss:1.816566	eval-mlogloss:1.834904
[4]	train-mlogloss:1.743812	eval-mlogloss:1.767241
[5]	train-mlogloss:1.676325	eval-mlogloss:1.703771
[6]	train-mlogloss:1.617723	eval-mlogloss:1.649438
[7]	train-mlogloss:1.562347	eval-mlogloss:1.598392
[8]	train-mlogloss:1.506026	eval-mlogloss:1.545712
[9]	train-mlogloss:1.455077	eval-mlogloss:1.497671
[10]	train-mlogloss:1.409737	eval-mlogloss:1.455635
[11]	train-mlogloss:1.366833	eval-mlogloss:1.415593
[12]	train-mlogloss:1.326972	eval-mlogloss:1.378905
[13]	train-mlogloss:1.288583	eval-mlogloss:1.343544
[14]	train-mlogloss:1.252429	eval-mlogloss:1.310498
[15]	train-mlogloss:1.216337	eval-mlogloss:1.277055
[16]	train-mlogloss:1.184735	eval-mlogloss:1.247855
[17]	train-mlogloss:1.152120	eval-mlogloss:1.218066
[18]	train-mlog